# Analysis of Vivino data 🍷
___

In [ ]:
# Import required libraries
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect(r"./db/vivino.db")

# Load CSV data into Pandas DataFrame
# sales_data = pd.read_csv("./db/vivino.db")
# Write the data to a sqlite table
# sales_data.to_sql("vivino", conn, if_exists="replace", index=False)

# Create a cursor object
cur = conn.cursor()


### 1/7 - We want to highlight 10 wines to increase our sales. Which ones should we choose and why?
___


### 2/7 -  We have a limited marketing budget for this year. Which country should we prioritise and why?
___

In the countries table we have different bits of information which could help us forma a decision,
however taking some rations would probably be the best solution. Here we compare:

- the average number of wines per user per country
- the average number of wineries per user per country 

We could interpet these results to say that the higher the ratio, the more likely a user is to encounter 

In [22]:
# Ratio other way around, percentages, cast as float, add where cluase for numerator = 0

query = """
        SELECT
            *,
            users_count/wines_count AS n_wines_per_user,
            users_count/wineries_count AS n_wineries_per_user
        FROM countries
        ORDER BY n_wines_per_user asc;
            """
countries_df = pd.read_sql_query(query, conn)
countries_df


,code,name,regions_count,users_count,wines_count,wineries_count,n_wines_per_user,n_wineries_per_user
0,md,Moldavie,9,13583,5055,418,2,32
1,hu,Hongrie,29,102235,16605,1923,6,53
2,cl,Chili,41,326757,41191,5785,7,56
3,za,Afrique du Sud,112,269649,30857,4227,8,63
4,gr,Grèce,140,95693,9581,1294,9,73
5,hr,Croatie,22,64223,5880,980,10,65
6,au,Australie,120,1022965,90954,13946,11,73
7,fr,France,1306,5973301,422503,67553,14,88
8,it,Italie,563,4270717,274658,42399,15,100
9,de,Allemagne,236,2549989,164533,13643,15,186


In [21]:
query = """
        SELECT
            name AS Country,
            users_count/wines_count AS n_wines_per_user,
            users_count/wineries_count AS n_wineries_per_user
        FROM countries
        ORDER BY n_wines_per_user desc;
            """
countries_df = pd.read_sql_query(query, conn)
countries_df


,Country,n_wines_per_user,n_wineries_per_user
0,États-Unis,60,436
1,Suisse,47,416
2,Roumanie,33,332
3,Portugal,28,192
4,Israël,27,284
5,Espagne,22,125
6,Argentine,17,117
7,Italie,15,100
8,Allemagne,15,186
9,France,14,88


In [19]:
import seaborn as sns

sns.lineplot(
    x="name", y=[["n_wines_per_user", "n_wineries_per_user"]], data=countries_df
)

ValueError: Length of list vectors must match length of `data` when both are used, but `data` has length 17 and the vector passed to `y` has length 1.

### 5/7 - We would like to select wines that are easy to find all over the world. Find the top 3 most common grapes all over the world and for each grape, give us the the 5 best rated wines.
___

First we find the most common grapes in the world. There are two methods which can be used:
1. We assume that the number of wines (wines_cout in most_used_grapes_per_country) is representative of the number of grapes.
2. We count the number of countries that have a specific type of grape


We went with the first option

In [11]:
query = """
        select
            grapes.name as grape_name,
            AVG(most_used_grapes_per_country.wines_count) as wines_count
        from most_used_grapes_per_country
        join grapes on most_used_grapes_per_country.grape_id = grapes.id
        group by grape_name
        order by wines_count desc
        limit 3;
            """
df = pd.read_sql_query(query, conn)
df


,grape_name,wines_count
0,Cabernet Sauvignon,801751.0
1,Chardonnay,604208.0
2,Pinot Noir,572334.0


Now we look at the top 5 wines that have the highest rating per type of grape

In [13]:
query = """
SELECT
    subquery.wine_name,
    subquery.ratings_average,
    subquery.grape_name
FROM
    (
        SELECT
            wines.name AS wine_name,
            wines.ratings_average,
            grapes.name AS grape_name,
            ROW_NUMBER() OVER (PARTITION BY grapes.name ORDER BY wines.ratings_average DESC) AS row_num
        FROM
            countries
            JOIN regions ON countries.code = regions.country_code
            JOIN wines ON regions.id = wines.region_id
            JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
            JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
        WHERE
            grapes.name IN ('Cabernet Sauvignon', 'Chardonnay', 'Pinot Noir')
    ) AS subquery
WHERE
    subquery.row_num <= 5
ORDER BY
    subquery.grape_name,
    subquery.ratings_average DESC
            """
df = pd.read_sql_query(query, conn)
df


,wine_name,ratings_average,grape_name
0,Cabernet Sauvignon,4.8,Cabernet Sauvignon
1,Mágico,4.8,Cabernet Sauvignon
2,IX Estate Red,4.7,Cabernet Sauvignon
3,Special Selection Cabernet Sauvignon,4.7,Cabernet Sauvignon
4,Unico Reserva Especial Edición,4.7,Cabernet Sauvignon
5,Amarone della Valpolicella Classico Riserva,4.8,Chardonnay
6,Cabernet Sauvignon,4.8,Chardonnay
7,Fratini Bolgheri Superiore,4.8,Chardonnay
8,Cristal Rosé Vinothèque,4.8,Chardonnay
9,Batard-Montrachet Grand Cru,4.8,Chardonnay


Some of these results are not very imformative (ex. row 0), so we attempt to add a winery category

In [15]:
query = """
SELECT
    subquery.wine_name,
    subquery.ratings_average,
    subquery.grape_name,
    subquery.winery_name
FROM
    (
        SELECT
            wines.name AS wine_name,
            wines.ratings_average,
            grapes.name AS grape_name,
            wineries.name AS winery_name,
            ROW_NUMBER() OVER (PARTITION BY grapes.name ORDER BY wines.ratings_average DESC) AS row_num
        FROM
            countries
            JOIN regions ON countries.code = regions.country_code
            JOIN wines ON regions.id = wines.region_id
            JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
            JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
            JOIN wineries ON wines.winery_id = wineries.id
        WHERE
            grapes.name IN ('Cabernet Sauvignon', 'Chardonnay', 'Pinot Noir')
    ) AS subquery
WHERE
    subquery.row_num <= 5
ORDER BY
    subquery.grape_name,
    subquery.ratings_average DESC
            """
df = pd.read_sql_query(query, conn)
df


,wine_name,ratings_average,grape_name,winery_name
0,The Armagh Shiraz,4.5,Cabernet Sauvignon,Tenuta Tignanello 'Solaia'
1,Lion Tamer Napa Valley Red Blend,4.3,Cabernet Sauvignon,Siepi
2,The Armagh Shiraz,4.5,Chardonnay,Tenuta Tignanello 'Solaia'
3,Raut Lagrein,4.5,Chardonnay,Corte di Cama Sforzato di Valtellina
4,Lion Tamer Napa Valley Red Blend,4.3,Chardonnay,Siepi
5,Lion Tamer Napa Valley Red Blend,4.3,Pinot Noir,Siepi


Unfortunately, this reduces our dataset drastically, and so cannot be used. Even if we were to do an outer join, none of our top 5s have a winery specified, meaning we have to come up with a different solution.